In [ ]:
from pennylane import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import pennylane as qml
import pandas as pd
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

In [ ]:
def powerFaultData(method = 'pca'):

	if method == 'pca':
		powerData = pd.read_csv('/Users/digvijay/Developer/MasterThesis/master-thesis-quantumkernelclassification/Code/DR_data/pca3.csv')
		
		X = []
		Y = []

		X = powerData[['pc1', 'pc2', 'pc3']]
		Y = powerData[['fault']]
		Y = np.asarray(Y).astype(int)

		xTrain, xTest, yTrain, yTest = train_test_split(X, Y, test_size = 0.95, shuffle=True, random_state=42)
		xTrain = preprocessing.normalize(xTrain)
		xTest = preprocessing.normalize(xTest)

		return xTrain, yTrain
	
	if method == 'fda':
		powerData = pd.read_csv('/Users/digvijay/Developer/MasterThesis/master-thesis-quantumkernelclassification/Code/DR_data/lda3.csv')
		
		X = []
		Y = []

		X = powerData[['LD1', 'LD2', 'LD3']]
		Y = powerData[['fault']]
		Y = np.asarray(Y).astype(int)

		xTrain, xTest, yTrain, yTest = train_test_split(X, Y, test_size = 0.95, shuffle=True, random_state=42)
		xTrain = preprocessing.normalize(xTrain)
		xTest = preprocessing.normalize(xTest)

		return xTrain, yTrain

In [ ]:
X, Y = powerFaultData('fda')

print("Data: ", X[0])
print("Label: ", Y[0])

In [ ]:
def target_alignment(
    X,
    Y,
    kernel,
    assume_normalized_kernel=False,
    rescale_class_labels=True,
):
    """Kernel-target alignment between kernel and labels."""

    K = qml.kernels.square_kernel_matrix(
        X,
        kernel,
        assume_normalized_kernel=assume_normalized_kernel,
    )

    if rescale_class_labels:
        nplus = np.count_nonzero(np.array(Y) == 1)
        nminus = len(Y) - nplus
        _Y = np.array([y / nplus if y == 1 else y / nminus for y in Y])
    else:
        _Y = np.array(Y)

    T = np.outer(_Y, _Y)
    inner_product = np.sum(K * T)
    norm = np.sqrt(np.sum(K * K) * np.sum(T * T))
    inner_product = inner_product / norm

    return inner_product

In [ ]:
dev = qml.device("default.qubit", wires=3, shots=None)
wires = dev.wires.tolist()

def layer(x, params, wires, i0=0, inc=1):
    """Building block of the embedding ansatz"""
    i = i0
    for j, wire in enumerate(wires):
        qml.Hadamard(wires=[wire])
        qml.RZ(x[i % len(x)], wires=[wire])
        #qml.RZ(x[i], wires=[wire])
        i += inc
        qml.RY(params[0, j], wires=[wire])

    qml.broadcast(unitary=qml.CRZ, pattern="ring", wires=wires, parameters=params[1])
    
def ansatz(x, params, wires):
    """The embedding ansatz"""
    for j, layer_params in enumerate(params):
        layer(x, layer_params, wires, i0=j * len(wires))
        
adjoint_ansatz = qml.adjoint(ansatz)

def random_params(num_wires, num_layers):
    """Generate random variational parameters in the shape for the ansatz."""
    return np.random.uniform(0, 2 * np.pi, (num_layers, 2, num_wires), requires_grad=True)

@qml.qnode(dev)
def kernel_circuit(x1, x2, params):
    ansatz(x1, params, wires=wires)
    adjoint_ansatz(x2, params, wires=wires)
    return qml.probs(wires=wires)

def kernel(x1, x2, params):
    return kernel_circuit(x1, x2, params)[0]

def accuracy(classifier, X, Y_target):
    return 1 - np.count_nonzero(classifier.predict(X) - Y_target) / len(Y_target)

In [ ]:
init_params = random_params(num_wires=3, num_layers=6)
print(init_params)
kernel_value = kernel(X[0], X[1], init_params)
print(f"The kernel value between the first and second datapoint is {kernel_value:.3f}")
print('---------------------------------------------------------------------------------------------')
print('Quantum Circuit: ')
drawer = qml.draw(kernel_circuit)
print(drawer(X[0], X[1], init_params))
print('---------------------------------------------------------------------------------------------')

In [ ]:
print("Kernel Alignment with Gradient Descent")

params = init_params
opt = qml.GradientDescentOptimizer(0.2)

for i in range(500):
    # Choose subset of datapoints to compute the KTA on.
    
    subset = np.random.choice(list(range(len(X))), 4)

    #print(type(subset))
    #print(type(Y))
    
    # Define the cost function for optimization
    cost = lambda _params: -target_alignment(
        X[subset],
        Y[subset],
        lambda x1, x2: kernel(x1, x2, _params),
        assume_normalized_kernel=True,
    )
    
    # Optimization step
    params = opt.step(cost, params)

    # Report the alignment on the full dataset every 50 steps.
    if (i + 1) % 10 == 0:
        current_alignment = target_alignment(
            X,
            Y,
            lambda x1, x2: kernel(x1, x2, params),
            assume_normalized_kernel=True,
        )
        print(f"Step {i+1} - Alignment = {current_alignment:.3f}")


trained_kernel = lambda x1, x2: kernel(x1, x2, params)
trained_kernel_matrix = lambda X1, X2: qml.kernels.kernel_matrix(X1, X2, trained_kernel)
svm_trained = SVC(kernel=trained_kernel_matrix).fit(X, Y)

accuracy_trained = accuracy(svm_trained, X, Y)
print(f"The accuracy of a kernel with trained parameters is {accuracy_trained:.3f}")

In [ ]:
def uncertinitySamplingSubset(X, svm_trained, subSize):
	
	probabilities = svm_trained.predict_proba(X)
	entropy = -np.sum(probabilities * np.log(probabilities), axis=1)
	selected_indices = np.argsort(entropy)[:subSize]

	return selected_indices

In [ ]:
print("Kernel Alignment with Gradient Descent with Active Learning")

trained_kernel = lambda x1, x2: kernel(x1, x2, params)
trained_kernel_matrix = lambda X1, X2: qml.kernels.kernel_matrix(X1, X2, trained_kernel)
svm_trained = SVC(kernel=trained_kernel_matrix, probability=True)
svm_trained.fit(X, Y)

accuracy_trained = accuracy(svm_trained, X, Y)
print(f"The accuracy of a kernel with trained parameters is {accuracy_trained:.3f}")

params = init_params
opt = qml.GradientDescentOptimizer(0.2)

for i in range(500):
    # Choose subset of datapoints to compute the KTA on.
    
    #subset = np.random.choice(list(range(len(X))), 4) 
    trained_kernel = lambda x1, x2: kernel(x1, x2, params)
    trained_kernel_matrix = lambda X1, X2: qml.kernels.kernel_matrix(X1, X2, trained_kernel)
    svm_trained = SVC(kernel=trained_kernel_matrix, probability=True).fit(X, Y)
    
    subset = uncertinitySamplingSubset(X, svm_trained=svm_trained, subSize=4)

    #print(type(subset))
    #print(type(Y))
    
    # Define the cost function for optimization
    cost = lambda _params: -target_alignment(
        X[subset],
        Y[subset],
        lambda x1, x2: kernel(x1, x2, _params),
        assume_normalized_kernel=True,
    )
    
    # Optimization step
    params = opt.step(cost, params)

    # Report the alignment on the full dataset every 50 steps.
    if (i + 1) % 10 == 0:
        current_alignment = target_alignment(
            X,
            Y,
            lambda x1, x2: kernel(x1, x2, params),
            assume_normalized_kernel=True,
        )
        print(f"Step {i+1} - Alignment = {current_alignment:.3f}")

trained_kernel = lambda x1, x2: kernel(x1, x2, params)
trained_kernel_matrix = lambda X1, X2: qml.kernels.kernel_matrix(X1, X2, trained_kernel)
svm_trained = SVC(kernel=trained_kernel_matrix).fit(X, Y)

accuracy_trained = accuracy(svm_trained, X, Y)
print(f"The accuracy of a kernel with trained parameters is {accuracy_trained:.3f}")